In [7]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

DATA_PATH = "../data/cifar-10-batches-py/"

def unpack(fname):
    import pickle
    with open(fname, "rb") as fin:
        dumped = pickle.load(fin, encoding="bytes")
    return dumped

def reshape(data):
    img = np.zeros((32, 32, 3), 'uint8')
    img[..., 0] = np.reshape(data[:1024], (32, 32))
    img[..., 1] = np.reshape(data[1024:2048], (32, 32))
    img[..., 2] = np.reshape(data[2048:3072], (32, 32))
    return img

batches_meta = unpack(DATA_PATH + "batches.meta")
data_batches = [
    unpack(DATA_PATH + "data_batch_" + str(i+1))
    for i in range(5)
]
test_batch = unpack(DATA_PATH + "test_batch")

# keras这个例子的输入需要每张图是3维数组，用上面的reshape函数应该刚好可以整成他们要的格式
for i in range(5):
    data_batches[i][b"data"] = np.array([reshape(img) for img in data_batches[i][b"data"]])
test_batch[b"data"] = np.array([reshape(img) for img in test_batch[b"data"]])

In [ ]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.
It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True ## 如果这里是true在训练的时候会自动对图像做一些微调增强训练效果
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5' ## 最后训练跑完模型会存到这里

# The data, split between train and test sets:
(x_train, y_train) = data_batches[0][b"data"], data_batches[0][b"labels"] ## 这里我做了处理，官方例子会从网上下载cifar10，我喂了第一个batch
(x_test, y_test) = test_batch[b"data"], test_batch[b"labels"]             ## 这里也一样
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (10000, 32, 32, 3)
10000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/100
313/313 [==============================] - 33s 104ms/step - loss: 2.1184 - acc: 0.2065 - val_loss: 1.9298 - val_acc: 0.3017
Epoch 2/100
313/313 [==============================] - 32s 102ms/step - loss: 1.8932 - acc: 0.3107 - val_loss: 1.7694 - val_acc: 0.3716
Epoch 3/100
313/313 [==============================] - 32s 102ms/step - loss: 1.7979 - acc: 0.3420 - val_loss: 1.6757 - val_acc: 0.3979
Epoch 4/100
313/313 [==============================] - 31s 100ms/step - loss: 1.7393 - acc: 0.3625 - val_loss: 1.6504 - val_acc: 0.3955
Epoch 5/100
313/313 [==============================] - 32s 103ms/step - loss: 1.6833 - acc: 0.3824 - val_loss: 1.5479 - val_acc: 0.4386
Epoch 6/100
313/313 [==============================] - 33s 105ms/step - loss: 1.6391 - acc: 0.4001 - val_loss: 1.5081 - val_acc: 0.4535
Epoch 7/100
313/313 [==============================] - 32s 103ms/step - loss

Epoch 59/100
313/313 [==============================] - 35s 110ms/step - loss: 0.9496 - acc: 0.6613 - val_loss: 0.9209 - val_acc: 0.6803
Epoch 60/100
313/313 [==============================] - 33s 106ms/step - loss: 0.9440 - acc: 0.6676 - val_loss: 0.9075 - val_acc: 0.6836
Epoch 61/100
313/313 [==============================] - 32s 103ms/step - loss: 0.9523 - acc: 0.6663 - val_loss: 0.8916 - val_acc: 0.6920
Epoch 62/100
313/313 [==============================] - 32s 101ms/step - loss: 0.9446 - acc: 0.6664 - val_loss: 0.9297 - val_acc: 0.6830
Epoch 63/100
313/313 [==============================] - 31s 101ms/step - loss: 0.9263 - acc: 0.6735 - val_loss: 0.9496 - val_acc: 0.6783
Epoch 64/100
313/313 [==============================] - 31s 99ms/step - loss: 0.9335 - acc: 0.6707 - val_loss: 0.9788 - val_acc: 0.6633
Epoch 65/100
313/313 [==============================] - 31s 100ms/step - loss: 0.9267 - acc: 0.6735 - val_loss: 0.8933 - val_acc: 0.6917
Epoch 66/100
313/313 [====================